<span style="display: inline-block; margin-right: 10px; vertical-align: middle; border: 1px solid #F8F8F8; border-radius: 8px; padding: 5px; background-color: #F8F8F8;">
    <a href="https://www.designsafe-ci.org/workspace/jupyter-lab-hpc-cuda-ds?appVersion=1.1.1" target="_parent" style="text-decoration: none;">
        <span style="font-family: Helvetica, sans-serif; font-size: 13px; color: #565656; margin-right: 5px; vertical-align: middle; font-weight: 600;">Open in</span>
        <img src="https://www.designsafe-ci.org/media/filer_public/2d/d3/2dd37fbf-289e-49cf-9c1a-879c864c4e17/nsf_nheri-ds.png" alt="Open in DesignSafe" style="width: 100px; height: auto; vertical-align: middle;">
    </a>
</span>

<span style="display: inline-block; margin-right: 10px; vertical-align: middle;">
    <a href="https://colab.research.google.com/github/NHERI-SimCenter/BrailsPlusPlus/blob/master/examples/image_processor/foundation_type_classifier.ipynb" target="_parent">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" style="width: 130px; height: auto; vertical-align: middle;"/>
    </a>
</span>

<span style="display: inline-block; vertical-align: middle; margin-top: 6px;">
    <a target="_blank" href="https://lightning.ai/new?repo_url=https%3A//github.com/NHERI-SimCenter/BrailsPlusPlus/blob/master/examples/image_processor/foundation_type_classifier.ipynb">
        <img src="https://pl-bolts-doc-images.s3.us-east-2.amazonaws.com/app-2/studio-badge.svg" alt="Open in Studio" style="width: 120px; height: auto; vertical-align: middle;"/>
    </a>
</span>

## **Install BRAILS++**
BRAILS++ is not yet available on PyPi. The following line installs the latest version from the GitHub repository using `pip`.

In [1]:
!pip install git+https://github.com/NHERI-SimCenter/BrailsPlusPlus

  Cloning https://github.com/bacetiner/BrailsPlusPlus to /tmp/pip-req-build-tkk_xs5z
  Running command git clone --filter=blob:none --quiet https://github.com/bacetiner/BrailsPlusPlus /tmp/pip-req-build-tkk_xs5z
  Resolved https://github.com/bacetiner/BrailsPlusPlus to commit ab696f6b0ede68c9488ed253403f87d8f7c5ff61
  Preparing metadata (setup.py) ... done
  Created wheel for brails: filename=brails-4.0-py3-none-any.whl size=2020188 sha256=b30ed1d18b406411da1079cff6b79a839320f0dbc5e762c435474d58f524448b
  Stored in directory: /tmp/pip-ephem-wheel-cache-ziaf1xg9/wheels/65/0b/aa/a9113e0557b48ca6dfe52e54ff2df5af4aa9945a3736b53797
Successfully built brails
Note: you may need to restart the kernel to use updated packages.


## **Import ImageSet and Importer Methods of BRAILS++**
BRAILS++ offers various Artificial Intelligence-powered modules for predicting building and infrastructure attributes from images, such as the `FoundationElevationClassifier` demonstrated in this example.

The `Importer` class provides the main methods to load these modules. To gather the images needed for attribute prediction, users can use the ImageSet class or BRAILS++ scrapers, which facilitate automated asset geometry and image retrieval. This example illustrates both methods for sourcing imagery for use with the `FoundationElevationClassifier` module.

In [2]:
from brails import Importer
from brails.types.image_set import ImageSet

## **Define Google API Key**
You need a Google API Key with Street View Static API enabled to download the Google Street View imagery required for this example. Please follow this [link](https://support.google.com/googleapi/answer/6158862?hl=en) for instructions on setting up a Google API key.

In [3]:
API_KEY = 'YOUR-API-KEY-HERE'

## **Load Street-Level Imagery: (Option 1) Download Imagery by Specifying the Region Name**
The `FoundationElevationClassifier` class identifies if a building is elevated or not from street-level imagery. In this option, BRAILS++ retrieves the required imagery by following the steps below.
1. <strong>Call the `RegionBoundary` class with the necessary details to specify the region of interest.</strong> In this example, street-level imagery of buildings in Fort Myers, FL, are downloaded. Therefore, the required information includes the `type` of information provided and the `data` itself, which are the `locationName` and `Fort Myers, FL`.

2. <strong>Download the footprint inventory for the `RegionBoundary` saved in `region_boundary_object` by downloading the OpenStreetMap footprint data through the `OSM_FootprintScraper`.</strong> The only information required to initialize `OSM_FootprintScraper` is the desired output unit for `length`, which, in this case, is `ft`. The output of `OSM_FootprintScraper` is an `AssetInventory` saved in `ft_myers_fp`.

3. <strong>Get a randomly selected `50` building subset of the footprint inventory obtained using `OSM_FootprintScraper`. </strong> This subset is obtained using the `get_random_sample` method of an `AssetInventory` object and is stored in `ft_myers_fp_subset`. For this example, the random seed is set to `5` for reproducibility of the results.

4. <strong>Get the street-level imagery for the selected subset using `get_images` method of `GoogleStreetview` module.</strong> `get_images` requires two inputs 1) AssetInventory for which the images will be retrieved (in this example, `ft_myers_fp_subset`) and 2) the path to the folder where the retrieved images will be saved, which in this case is `tmp/street/`.

In [4]:
# Select a region and create its RegionBoundary:
importer = Importer()

region_data = {"type": "locationName",
               "data": "Fort Myers Beach, FL"}
region_boundary_class = importer.get_class("RegionBoundary")
region_boundary_object = region_boundary_class(region_data)

osm_fp_class = importer.get_class("OSM_FootprintScraper")
osm_fp_data = {"length": "ft"}
footprint_scraper = osm_fp_class(osm_fp_data)
ft_myers_fp = footprint_scraper.get_footprints(region_boundary_object)

# Subsample from the extracted assets to keep the image downloading step quick.
# Here, we are randomly sampling 50 buildings using a random seed value of 5:
ft_myers_fp_subset = ft_myers_fp.get_random_sample(50, 5)

# Get aerial imagery for the selected subset using GoogleSatellite:
google_street_class = importer.get_class("GoogleStreetview")
google_street = google_street_class({'apiKey': API_KEY})
images_street = google_street.get_images(
    ft_myers_fp_subset, "tmp/street/")


Searching for Fort Myers Beach, FL...
Found Fort Myers Beach, Lee County, Florida, 33931, United States

Found a total of 2768 building footprints in Fort Myers Beach


INFO:brails.scrapers.google_streetview.google_streetview:
Images will be saved to: /home/bacetiner/Documents/BrailsPlusPlus/examples/image_processor/tmp/street

Obtaining street-level imagery: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:06<00:00,  7.19it/s]


## **Predict If Building Foundations Are Elevated Using FoundationElevationClassifier Module**
`FoundationElevationClassifier` includes a pre-trained ResNet-50 model that has been trained on a custom dataset of 8,000 images. This model is capable of classifying street-level imagery of buildings into two classes: `Elevated`, and `Non-elevated`.

In [5]:
importer = Importer()
found_elev_classifier_class = importer.get_class('FoundationElevationClassifier')
found_elev_classifier_class = found_elev_classifier_class()
predictions = found_elev_classifier_class.predict(images_street)

/home/bacetiner/anaconda3/envs/brails_plus/lib/python3.11/site-packages/brails/processors/foundation_classifier/foundation_classifier.py:343: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimen

Missing or unexpected keys: ['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.conv2.weight', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.conv3.weight', 'layer1.0.bn3.weight', 'layer1.0.bn3.bias', 'layer1.0.bn3.running_mean', 'layer1.0.bn3.running_var', 'layer1.0.downsample.0.weight', 'layer1.0.downsample.1.weight', 'layer1.0.downsample.1.bias', 'layer1.0.downsample.1.running_mean', 'layer1.0.downsample.1.running_var', 'layer1.1.conv1.weight', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.conv2.weight', 'layer1.1.bn2.weight', 'layer1.1.bn2.bias', 'layer1.1.bn2.running_mean', 'layer1.1.bn2.running_var', 'layer1.1.conv3.weight', 'layer1.1.bn3.weight', 'layer1.1.bn3.bias', 'layer1.1.bn3.running_

## **Show Prediction Results**

In [6]:
print(predictions)

{212: 'Elevated', 417: 'Elevated', 21: 'Elevated', 653: 'Elevated', 745: 'Elevated', 744: 'Elevated', 642: 'Non-elevated', 2531: 'Elevated', 2658: 'Non-elevated', 2562: 'Non-elevated', 2529: 'Non-elevated', 53: 'Elevated', 2227: 'Elevated', 2350: 'Elevated', 806: 'Elevated', 2208: 'Non-elevated', 2171: 'Elevated', 1907: 'Non-elevated', 7: 'Elevated', 1921: 'Elevated', 1284: 'Non-elevated', 681: 'Elevated', 679: 'Elevated', 882: 'Elevated', 1009: 'Non-elevated', 838: 'Elevated', 568: 'Elevated', 1046: 'Non-elevated', 887: 'Elevated', 857: 'Non-elevated', 463: 'Elevated', 1184: 'Non-elevated', 1020: 'Elevated', 1468: 'Non-elevated', 118: 'Elevated', 1144: 'Non-elevated', 1223: 'Non-elevated', 1822: 'Elevated', 541: 'Elevated', 814: 'Non-elevated', 1021: 'Elevated', 518: 'Elevated', 1671: 'Non-elevated', 1569: 'Elevated', 1522: 'Elevated', 1559: 'Elevated', 1595: 'Non-elevated'}
